This version uses Milvus through Docker Compose so you must have Docker installed to run this notebook (Milvus is spun up via `docker compose up -d` as shown in the block below)

In [649]:
# ! pip install -qU pymilvus langchain sentence-transformers tiktoken octoai-sdk openai
# !docker compose up -d

In [650]:
from dotenv import load_dotenv
import os



load_dotenv()
# %env OCTOAI_API_KEY = 'alnvnipa4gjj'
# %env OCTOAI_API_TOKEN = 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IjNkMjMzOTQ5In0.eyJzdWIiOiIwNTA3Y2IwOC03ODM2LTRhMTItOTA2NS0wMjczYTUxMjI5OWUiLCJ0eXBlIjoidXNlckFjY2Vzc1Rva2VuIiwidGVuYW50SWQiOiIxODg5MTdhYS05OTgyLTRkYmUtOGM2OC1lZGU2NzZjNDMzYTgiLCJ1c2VySWQiOiI5MzRiMWI2OC0yYTY3LTQwYjYtOGY5OS05N2IzMzgzZWE4YmMiLCJhcHBsaWNhdGlvbklkIjoiYTkyNmZlYmQtMjFlYS00ODdiLTg1ZjUtMzQ5NDA5N2VjODMzIiwicm9sZXMiOlsiRkVUQ0gtUk9MRVMtQlktQVBJIl0sInBlcm1pc3Npb25zIjpbIkZFVENILVBFUk1JU1NJT05TLUJZLUFQSSJdLCJhdWQiOiIzZDIzMzk0OS1hMmZiLTRhYjAtYjdlYy00NmY2MjU1YzUxMGUiLCJpc3MiOiJodHRwczovL2lkZW50aXR5Lm9jdG9tbC5haSIsImlhdCI6MTcxNDkzMDQ0Mn0.SoasYiHlD-7cX4ubQk5Um7RwdyHD_NUs74T33aGruTsR6Svaaoecr8uxWHw5QbJH6wUmYy1--ulhKUgABfd6RnN610d5C0lEMTKaF7bDcg_yI-B4i67d9E5N2zJck1yeDIW9swJcLfEWS6XYu2cE34wrP__R1ss8OxA7OwoMBnkpOaMya9u11eJaMyIPidavqol5byssvpozn-gM3iEOogVPaKP5MFXyi-27wmosQ6frgTJYyQ1CpOs2-XuKEwko6MJFiK1ezr_TiBZ3LH7uYO1m2FbtrltrEQQCR9ZeWUZJdNjRLdkZqhA3-chJQR6DMwfO0eq2MYlGJ4lbHMjD4w'


OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]


In [651]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct-fp16",
        max_tokens=200,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

/Users/sangwonan/hackerthon/.venv/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [652]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [653]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [654]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [655]:
files = os.listdir("../city_data")
# files = os.listdir("../terms_and_condition_data")

In [656]:
files

['Signal.txt',
 'Epic Games.txt',
 'moBILET.txt',
 'Nike.txt',
 'EliteSingles.txt',
 'Baidu AI Cloud.txt',
 'Tidal.txt',
 'Glovo.txt',
 'Viber.txt',
 'GG.txt',
 'Tinder.txt',
 'vod.mdag.pl.txt',
 'Vinted.txt',
 'Meta.txt',
 'Dropbox.txt',
 'Microsoft.txt',
 'Adobe.txt',
 'DingTalk.txt',
 'KAYAK.txt',
 'Pyszne.pl.txt',
 'Maya.txt',
 'Steam.txt',
 'TripAdvisor.txt',
 'OVH.txt',
 'Strava.txt',
 'DoorDash.txt',
 'FollowMyHealth.txt',
 'PAYBACK.txt',
 'Spotify.txt',
 'Cisco.txt',
 'Blik.txt',
 'Toggl.txt',
 'Supercell.txt',
 'Hotels.com.txt',
 'Telegram.txt',
 'Gett.txt',
 'Niantic.txt',
 'Bolt.txt',
 'Zenly.txt',
 'HBO GO.txt',
 'Zalando.txt',
 'Booking.com.txt',
 'Krakowskie Smaki.txt',
 'ABEMA.txt',
 'Aliexpress.txt',
 'Zoom.txt',
 'Twitch.txt',
 'YouTube.txt',
 'FineDine.txt',
 'Oktawave.txt',
 'Yahoo.txt',
 'BlaBlaCar.txt',
 'MyFitnessPal.txt',
 'Line.txt',
 'Crypto.com.txt',
 'iTaxi.txt',
 'Pinterest.txt',
 'Google.txt',
 'MyTaxi.txt',
 'Ubisoft.txt',
 'Sympatia.pl.txt',
 'Sleep Cycle

In [657]:
file_texts = []

In [658]:
for file in files:
    with open(f"../city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=64, separator='\n'
    )
    
    texts = text_splitter.split_text(file_text)
    # texts = file_text.split('\n')
    # print(texts)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))
        # print(len(chunked_text))


Created a chunk of size 556, which is longer than the specified 512
Created a chunk of size 684, which is longer than the specified 512
Created a chunk of size 529, which is longer than the specified 512
Created a chunk of size 858, which is longer than the specified 512
Created a chunk of size 545, which is longer than the specified 512
Created a chunk of size 583, which is longer than the specified 512
Created a chunk of size 647, which is longer than the specified 512
Created a chunk of size 582, which is longer than the specified 512
Created a chunk of size 548, which is longer than the specified 512
Created a chunk of size 561, which is longer than the specified 512
Created a chunk of size 545, which is longer than the specified 512
Created a chunk of size 607, which is longer than the specified 512
Created a chunk of size 865, which is longer than the specified 512
Created a chunk of size 616, which is longer than the specified 512
Created a chunk of size 1649, which is longer th

In [659]:
# For the first run
# # # 
# vector_store = Milvus.from_documents(
#     file_texts,
#     embedding=embeddings,
#     connection_args={"host": "localhost", "port": 19530},
#     collection_name="city_data"
# )

# if you already have the data you need stored in Milvus
# vector_store = Milvus(
#     embedding_function=embeddings,
#     connection_args={"host": "localhost", "port": 19530},
#     collection_name="city_data"
# )

In [660]:
retriever = vector_store.as_retriever()

In [661]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [662]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [663]:
response1 = chain.invoke("Summarize privacy terms for Adobe")
response2 = chain.invoke("I am going to use Youtube, Does Youtube have any rights to use my video?")
response3 = chain.invoke("Can you smmarize service terms for Yahoo?")

In [664]:
response3

" Yahoo's service terms include a liability limit for disputes, which is the amount paid for the services. Users must not violate laws or the Community Guidelines, and should be of the minimum age to use the services. Fee-based services require accurate billing information and are subject to third-party terms."

In [665]:
response2


' Yes, when you upload a video to YouTube, you grant them a license to use your video for operating, promoting, and improving their service. This includes the right to reproduce, distribute, modify, display, and perform your video. Other users also receive a license to access your video through the Service. These licenses terminate when you remove your video, except where the law requires otherwise.'